<a href="https://colab.research.google.com/github/ywChen-NTUST/DS_hw2/blob/main/DS_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load file into colab workspace

In [ ]:
!gdown --id "1gLpMZ_sdjaSk6C5RkqQiLEVyvAD9GZv9" --output "data.csv"
!gdown --id "1gUSSIntugJwDu_pAUdEI9GghRUSEMCbv" --output "test.csv"

Downloading...
From: https://drive.google.com/uc?id=1gLpMZ_sdjaSk6C5RkqQiLEVyvAD9GZv9
To: /content/data.csv
100% 370k/370k [00:00<00:00, 44.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gUSSIntugJwDu_pAUdEI9GghRUSEMCbv
To: /content/test.csv
100% 5.49k/5.49k [00:00<00:00, 9.41MB/s]


In [ ]:
!ls

data.csv  sample_data  test.csv


# Preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("data.csv")
data.head()

,id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19
0,0,86,140,9,0.0,0.0,5.444444,4.768726,3.055556,2.678447,9.925926,6.000000,15.111111,8.666667,-11.777778,15.555555,-3.777778,15.111111,0.627944,-2.417818
1,1,34,93,9,0.0,0.0,0.555556,0.272165,0.388889,0.389682,14.814815,10.444445,23.444445,10.555555,-13.111111,25.888890,-12.777778,23.444445,0.559179,-2.102707
2,2,134,148,9,0.0,0.0,0.111111,0.172133,0.055556,0.136083,0.037037,0.000000,0.111111,0.000000,-0.111111,0.222222,-0.111111,0.111111,0.111111,-2.094395
3,3,199,144,9,0.0,0.0,0.333333,0.516398,0.333333,0.365148,0.444444,0.111111,1.111111,0.111111,-1.000000,2.000000,-1.000000,1.111111,0.740741,-2.094395
4,4,197,236,9,0.0,0.0,2.444444,6.829628,3.333333,7.599998,16.074074,13.111111,16.666668,18.444445,-8.888889,1.777778,7.111111,18.555555,0.292729,2.789800


In [ ]:
from sklearn import preprocessing

In [ ]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)

# Train model

## ~~Basic model (kMeans)~~

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeansModel = KMeans(n_clusters=9, init="k-means++", max_iter=300, verbose=1, random_state=29)
labels = kmeansModel.fit_predict(data)

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 520.6088289002676
start iteration
done sorting
end inner loop
Iteration 1, inertia 497.39643367937447
start iteration
done sorting
end inner loop
Iteration 2, inertia 489.59569815707783
start iteration
done sorting
end inner loop
Iteration 3, inertia 488.1133381956975
start iteration
done sorting
end inner loop
Iteration 4, inertia 487.2431032851391
start iteration
done sorting
end inner loop
Iteration 5, inertia 486.6954760498242
start iteration
done sorting
end inner loop
Iteration 6, inertia 486.17485703437296
start iteration
done sorting
end inner loop
Iteration 7, inertia 485.3814647227572
start iteration
done sorting
end inner loop
Iteration 8, inertia 483.9552620676668
start iteration
done sorting
end inner loop
Iteration 9, inertia 482.7170456603813
start iteration
done sorting
end inner loop
Iteration 10, inertia 481.8480487518778
start iteration
done sorting
end inner loop
Iteration 11, i

## Hierarchical model (AgglomerativeClustering)

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
hierarchicalModel = AgglomerativeClustering(n_clusters=9, affinity='euclidean', memory="./model_cache/", linkage="ward") #n_cluster=9 affinity='euclidean' linkage="ward"
labels = hierarchicalModel.fit_predict(data)

## ~~Density base model (DBSCAN)~~

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
densityModel = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
labels = densityModel.fit_predict(data)

## ~~DBSCAN + Hierarchical~~

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

In [ ]:
hierarchicalModel = AgglomerativeClustering(n_clusters=9, affinity='euclidean', memory="./model_cache/", linkage="ward")
labels_sup = hierarchicalModel.fit_predict(data)

In [ ]:
densityModel = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
labels_main = densityModel.fit_predict(data)

In [ ]:
assert (labels.shape[0] == labels_sup.shape[0])

labels = labels_main.copy()

for index in range(labels.shape[0]):
  if(labels[index] == -1):
    group_sup = labels_sup[index]

    same_group_i = 0
    for sup_index in range(labels_sup.shape[0]):
      if (labels_sup[sup_index] == group_sup and sup_index != index):
        same_group_i = sup_index
        break

    print(index, same_group_i)
    labels[index] = labels[same_group_i]

31 26
42 26
57 4
134 26
349 26
426 26
475 26
531 4
626 26
660 26
695 26
736 26
758 26
821 9
834 21
857 26
876 21
912 26
939 26
955 26
1085 26
1235 26
1308 21
1443 4
1468 21
1637 26
1658 40
1659 21
1710 26
1748 9
1880 9
1919 6
1926 26
1960 26
1982 26


In [ ]:
labels.tolist().count(-1)

0

# Apply model

In [ ]:
cmp = pd.read_csv("test.csv")

In [ ]:
result = []
for i in range(cmp.shape[0]):
  x1 = cmp.iloc[i]["0"]
  x2 = cmp.iloc[i]["1"]

  result.append(labels[int(x1)]==labels[int(x2)] * 1)

# Store result

In [ ]:
with open("result.csv", 'w') as fh:
  fh.write("index,ans"+"\n")
  for i in range(cmp.shape[0]):
    fh.write(str(i)+","+str(int(result[i]))+"\n")

In [ ]:
from google.colab import files
files.download('result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>